In [ ]:
input_file = "input.txt"
import pandas as pd
import numpy as np
from itertools import product

with open(input_file) as file:
    lines = file.readlines()
    lines = list(map(lambda x: list(x.strip()), lines))
    df = pd.DataFrame(lines)
    
df=df[df != "."]
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN


In [24]:
indizes = df.stack().index.values
antennas_per_frequency = {}

for index in indizes:
    row, column = index
    frequency = df.iloc[row, column]
    if frequency in antennas_per_frequency:
        antennas_per_frequency[frequency].append(index)
    else: antennas_per_frequency[frequency] = [index]
    
antennas_per_frequency

def check_antinode_inbounds(antinode, shape):
    row, column = antinode
    shape_rows, shape_columns = shape
    return row >= 0 and column >= 0 and row < shape_rows and column < shape_columns

def get_connections(locations):
    connections = []
    for i in range(0, len(locations)):
        for j in range(0, i):
            connections.append((locations[i], locations[j]))
    return connections

def get_antinodes_of_connection_part_1(row_1, row_2, row_diff, column_1, column_2, column_diff):
        temp_antinodes =[(row_1-row_diff, column_1-column_diff),(row_2+row_diff, column_2+column_diff)]
        return list(filter(lambda x: check_antinode_inbounds(x, df.shape), temp_antinodes))

def walk_until_out_of_bounds(starting_point, row_diff, column_diff):
    result = []
    row, column = starting_point
    in_bounds = True
    while in_bounds:
        next_point = (row + row_diff, column + column_diff)
        if check_antinode_inbounds(next_point, df.shape):
            result.append(next_point)
            row, column = next_point
        else: in_bounds = False
    return result


def get_antinodes_of_connection_part_2(row_1, row_2, row_diff, column_1, column_2, column_diff):
    result = walk_until_out_of_bounds((row_1, column_1), -row_diff, -column_diff) + walk_until_out_of_bounds((row_2, column_2), row_diff, column_diff)
    return result

def get_antinodes_of_frequency(locations, part):
    antinodes = []
    for connection in get_connections(locations):
        point_1, point_2 = connection
        if point_1 == point_2: continue
        row_1, column_1 = point_1
        row_2, column_2 = point_2
        row_diff = row_2 - row_1
        column_diff = column_2 - column_1
        if(part == 1):
            antinodes += get_antinodes_of_connection_part_1(row_1, row_2, row_diff, column_1, column_2, column_diff)
        else:
            antinodes += get_antinodes_of_connection_part_2(row_1, row_2, row_diff, column_1, column_2, column_diff)
    return antinodes
        

part=2
antinodes=[]
for index, locations in enumerate(antennas_per_frequency.values()):
    # print(str(index) + " of " + str(len(antennas_per_frequency.values())))
    antinodes += get_antinodes_of_frequency(locations,part)
    if part == 2:
        antinodes += locations
    
len(set(antinodes))

34